### Instalaciones

#### Convertidor de divisas

In [ ]:
pip install CurrencyConverter

##### Tablas dinámicas

In [ ]:
pip install itables

#### Encontrar NA graficamente

In [ ]:
pip install missingno

### Librerías

In [ ]:
import pandas as pd
from currency_converter import CurrencyConverter
import matplotlib.pylab as plt
import plotly.express as px
from pywaffle import Waffle
from pivottablejs import pivot_ui
import seaborn as sns
import numpy as np
import plotly.figure_factory as ff
import dataframe_image as dfi

In [ ]:
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

In [ ]:
import missingno as msno

%matplotlib inline #render plots within jupyter notebook

#### Importando CSV 2021

In [ ]:
directory2021="Contratos2021_210219090543 2.xlsx"
contratos2021=pd.read_excel(directory2021)

#### Importando CSV 2020

In [ ]:
directory2020="Contratos2020_210223083001.xlsx"
contratos2020=pd.read_excel(directory2020)

In [ ]:
#Fucionando frames
frames = [contratos2021, contratos2020]
contratos=pd.concat(frames)

#### Valores nulos (gráfica)

In [ ]:
msno.matrix(contratos)


### Limpiando base de datos

In [7]:
# Columnas
columnas=[]

for col in contratos.columns: 
    append.columnas(col)

print(columnas)

In [ ]:
contratos['Proveedor o contratista'] = contratos['Proveedor o contratista'].str.upper() 

### Convirtiendo divisas

In [ ]:
#Precios divisas
c = CurrencyConverter()
currencies=['USD', 'CAD', 'EUR', 'GBP', 'JPY']
for m in currencies:
    print(m, c.convert(1, m, 'MXN'))

In [ ]:
USD=c.convert(1, 'USD', 'MXN')
CAD=c.convert(1, 'CAD', 'MXN')
EUR=c.convert(1, 'EUR', 'MXN')
GBP=c.convert(1, 'GBP', 'MXN')
JPY=c.convert(1, 'JPY', 'MXN')

In [ ]:
contratos["Importe"]=contratos["Importe del contrato"]

contratos.loc[contratos['Moneda del contrato'] == "USD" , 'Importe']=contratos.loc[contratos['Moneda del contrato'] == "USD" , "Importe del contrato"]*USD

contratos.loc[contratos['Moneda del contrato'] == "CAD" , 'Importe']=contratos.loc[contratos['Moneda del contrato'] == "CAD" , 'Importe del contrato']*CAD
    
contratos.loc[contratos['Moneda del contrato'] == "EUR" , 'Importe']=contratos.loc[contratos['Moneda del contrato'] == "EUR" , 'Importe del contrato']*EUR
    
contratos.loc[contratos['Moneda del contrato'] == "GBP" , 'Importe']=contratos.loc[contratos['Moneda del contrato'] == "GBP" , 'Importe del contrato']*GBP
    
contratos.loc[contratos['Moneda del contrato'] == "JPY" , 'Importe']=contratos.loc[contratos['Moneda del contrato'] == "JPY" , 'Importe del contrato']*JPY

print('Done')

### Filtro de contratos

In [ ]:
#x=contratos[np.logical_and(contratos['Título del expediente'].str.contains('DIFUSIÓN', na=False), 
                            #contratos['Siglas de la Institución']=='SEDENA')]
#x=contratos[contratos['Proveedor o contratista'].str.contains('Proveedor o contratista', na=False)]
#x=contratos[contratos['Importe']==25849460178.2]

x[['Proveedor o contratista', 'Siglas de la Institución', 'Importe', 'Título del expediente', 'Dirección del anuncio', 'Tipo de procedimiento']]

In [ ]:
caros=contratos[np.logical_and(contratos['Importe']>=0, contratos['Siglas de la Institución']=='SRE')]
caros.sort_values(by='Importe', ascending=True, inplace=True)
caros[['Proveedor o contratista', 'Siglas de la Institución','Título del expediente', 'Importe', 'Fecha de publicación', 'Dirección del anuncio', 'Tipo de procedimiento']]

### Contratos más costosos

In [ ]:
caros=contratos[np.logical_and(contratos['Importe']>=0, contratos['Siglas de la Institución']=='SRE')]
caros.sort_values(by='Importe', ascending=True, inplace=True)
caros[['Proveedor o contratista', 'Siglas de la Institución', 'Importe', 'Título del expediente', 'Dirección del anuncio', 'Tipo de procedimiento']].head(20)

### Gráfica de Waffle

In [ ]:
procesos=dict(contratos['Tipo de procedimiento'].value_counts())
total_procesos=sum(procesos.values())
print(procesos, "\n")
print(total_procesos)

In [ ]:
for x,y in procesos.items():
    procesos[x] = float("{:.2f}".format((y/total_procesos)*100))
    print(x,procesos[x])


In [ ]:
fig = plt.figure(FigureClass=Waffle,
                 figsize=(10,10),
                 rows=10,
                 columns=30,
                 values=procesos,
                 title={'label': 'Figura 2\n ', 'loc': 'center', 'fontdict': {'fontsize': 16}},
                 colors=["#f12832", "#000c2e", "#94e2ba","#636363", "#9367aa",'#a2ff1c'],
                 labels=[f"{k} ({v}%)" for k, v in procesos.items()],
                 legend={'loc': (0, -0.5), 'ncol': 1, 'fontsize':12, 'framealpha': 2})

In [ ]:
pd.options.display.float_format = '{:.0f}'.format
procedimiento_importe=contratos.groupby("Tipo de procedimiento")["Importe"].sum().reset_index()
procedimiento_importe.sort_values(by='Importe', ascending=False, inplace=True)
procedimiento_importe.head(10)


In [ ]:
procesos=procedimiento_importe['Tipo de procedimiento'].tolist()
importe=procedimiento_importe['Importe'].tolist()

procesos_importe=dict(zip(procesos,importe))
print(procesos_importe)
#procesos_importe = dict(zip(procesos, importe))


In [ ]:
total_importe=sum(importe)
for x,y in procesos_importe.items():
    procesos_importe[x] = float("{:.2f}".format((y/total_importe)*100))
    print(x,procesos_importe[x])


### Acumulado por instituciones


In [ ]:
pd.options.display.float_format = '{:,}'.format
Institucion=contratos.groupby("Siglas de la Institución")["Importe"].sum().reset_index()
Institucion.sort_values(by='Importe', ascending=False, inplace=True)
Institucion.set_index('Siglas de la Institución', inplace=True)
Institucion

In [ ]:
pd.options.display.float_format = '{:,}'.format
Institucion_contratos=contratos.groupby("Siglas de la Institución")["Importe"].count().reset_index()
Institucion_contratos.sort_values(by='Importe', ascending=False, inplace=True)
Institucion_contratos.set_index('Siglas de la Institución', inplace=True)
Institucion_contratos.rename(columns={'Importe':'Número de contratos'}, inplace=True)
Institucion_contratos

### Proovedores y filtro por proovedor

In [ ]:
pd.set_option("display.max_rows", contratos.index.shape[0])
pd.options.display.float_format = '{:,}'.format
#proveedor=contratos[contratos['Siglas de la Institución']=='BIENESTAR'].groupby("Proveedor o contratista")["Importe"].sum().reset_index()
#proveedor=contratos[contratos['Siglas de la Institución']=='SCT']
proveedor=contratos.groupby("Proveedor o contratista")["Importe"].sum().reset_index()

proveedor.sort_values(by='Importe', ascending=False, inplace=True)
proveedor.set_index("Proveedor o contratista", inplace=True)

#proveedor.set_index('Proveedor o contratista', inplace=True)
proveedor

In [ ]:
pd.options.display.float_format = '{:,}'.format
proveedores_contratos=contratos.groupby('Proveedor o contratista')["Importe"].count().reset_index()
proveedores_contratos.sort_values(by='Importe', ascending=False, inplace=True)
proveedores_contratos.set_index("Proveedor o contratista", inplace=True)
proveedores_contratos.rename(columns={'Importe':'Número de contratos'}, inplace=True)
proveedores_contratos


### Microempresas

In [ ]:
pd.options.display.float_format = '{:,}'.format
contratos['Estratificación de la empresa'].fillna('NaN', inplace=True)

micro=contratos.groupby(['Proveedor o contratista', 'Estratificación de la empresa']).agg({'Importe':'sum'}).reset_index()
micro.sort_values(by='Importe', ascending=False, inplace=True)
micro['Estratificación de la empresa'].fillna('NaN', inplace=True)

micro.set_index("Proveedor o contratista", inplace=True)

micro=micro[np.logical_or(micro['Estratificación de la empresa'] == 'Micro', micro['Estratificación de la empresa'] == 'NaN')]

#proveedor.set_index('Proveedor o contratista', inplace=True)
micro